# Convolutional Neural Network for 1NTPPP

In [1]:
import numpy as np

from archive_parser import archive_to_dataframe

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, InputLayer, Input, Reshape
from keras.utils import np_utils

from sklearn.model_selection import train_test_split

filename = "archive.dat"
df = archive_to_dataframe(filename)

X = np.stack(df["player"].values)
y = (df["entame"]//13).values
y = np.stack(np_utils.to_categorical(y))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

Using TensorFlow backend.


Basically the same as for Ziyed's FFNN, with a few imports added

In [2]:
size_flat = 52;
shape_full = (4, 13, 1)
num_of_chanels = 1
num_classes = 4

Some variables that will come useful later.
Note that shape_full will be used to shape our unidimensional vector of 52 bits in a 4 * 13 vector.

In [3]:
model = Sequential()
model.add(InputLayer(input_shape=(size_flat,)))

model.add(Reshape(shape_full))

We want to reshape the 52 unidimensional vector into a 4 * 13 one.

In [4]:
# First convolutional layer with ReLU-activation
model.add(Conv2D(kernel_size=4, strides=1, filters=26, padding='same',
                 activation='relu', name='layer_conv1'))

# Second convolutional layer with ReLU-activation
model.add(Conv2D(kernel_size=4, strides=1, filters=52, padding='same',
                 activation='relu', name='layer_conv2'))

# Third convolutional layer with ReLU-activation
model.add(Conv2D(kernel_size=4, strides=1, filters=52, padding='same',
                 activation='relu', name='layer_conv3'))

# Trying with a fourth layer
#model.add(Conv2D(kernel_size=2, strides=1, filters=52, padding='same',
#                 activation='relu', name='layer_conv4'))

Adding the convolutional layers. I tried with a fourth one but it won't change the overall performance.
Here are some of the parameters that can be changed in order to optimize the CNN : try changing the kernel_size and the number of filters in each layer. 

Best results so far have been obtained with : layer_conv1 (4, 26), layer_conv2 (4, 52), layer_conv3 (4, 52)

In [5]:
# Flatten the 4-rank output of the convolutional layers
# to 2-rank that can be input to a fully-connected / dense layer
model.add(Flatten())

# First fully-connected / dense layer with ReLU-activation
model.add(Dense(26, activation='relu'))

# Last fully-connected / dense layer with softmax-activation
model.add(Dense(num_classes, activation='softmax'))

Connecting to the final layers : one fully connected layer with 26 features (try and change that number to see its influence), and a final classification layer corresponding to the four colours.

In [6]:
epochs = 20
batch_size = 256

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train,
          y_train,
          epochs=epochs, batch_size=batch_size)

score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
print("Accuracy: ", score[1]*100)

Epoch 1/20
1978/1978 [==============================] - 1s - loss: 1.3649 - acc: 0.3241     
Epoch 2/20
1978/1978 [==============================] - 0s - loss: 1.2563 - acc: 0.4065     
Epoch 3/20
1978/1978 [==============================] - 0s - loss: 0.9958 - acc: 0.5875     
Epoch 4/20
1978/1978 [==============================] - 1s - loss: 0.8025 - acc: 0.6835     
Epoch 5/20
1978/1978 [==============================] - 0s - loss: 0.7350 - acc: 0.7113     
Epoch 6/20
1978/1978 [==============================] - 0s - loss: 0.6683 - acc: 0.7503     
Epoch 7/20
1978/1978 [==============================] - 0s - loss: 0.6490 - acc: 0.7528     
Epoch 8/20
1978/1978 [==============================] - 0s - loss: 0.6345 - acc: 0.7573     
Epoch 9/20
1978/1978 [==============================] - 1s - loss: 0.6058 - acc: 0.7801     
Epoch 10/20
1978/1978 [==============================] - 1s - loss: 0.5976 - acc: 0.7705     
Epoch 11/20
1978/1978 [==============================] - 1s - loss: 0

You can change the epochs and batch_size parameters too. The best I have found so far are the current parameters of 20 epochs on a 256 batch size. We obtain about 80% accuracy, which seems to be hard to beat, given that we test on human games and human don't always play the "best" lead.